### Imports

In [13]:
%pylab inline
from glob import glob
from nipype.interfaces import afni
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node
from os import makedirs
from os.path import abspath, join
import pandas as pd
import shutil


Populating the interactive namespace from numpy and matplotlib


### Experiment Variables

In [2]:
data_dir = '/mnt/OAK'
experiment_dir = '/home/ian/temp'
output_dir = '1stLevel'
working_dir = 'workingdir'

# list of subject identifiers
subject_list = ['sub-s358']

# list of task identifiers
task_list = ['stroop', 'ANT', 'CCTHot', 'DPX', 'twoByTwo']

# TR of functional images
TR = .68

regress_rt=True


# Set up Nodes

### Define helper functions

In [3]:
# helper function to create bunch
def subjectinfo(data_dir, subject_id, task, 
                regress_rt, inspect_inputs=False):
    
    from glob import glob
    from os.path import join
    import pandas as pd
    from nipype.interfaces.base import Bunch
    from utils.utils import get_contrasts, parse_EVs, process_confounds
    
    # strip "sub" from beginning of subject_id if provided
    subject_id = subject_id.replace('sub-','')
    
    ## Get the Events File
    
    # Read the TSV file and convert to pandas dataframe
    event_file = glob(join(data_dir,
                           'sub-%s' % subject_id,
                           '*', 'func',
                           '*%s*events.tsv' % task))[0]
    events_df = pd.read_csv(event_file,sep = '\t')

    ## Get the Confounds File (output of fmriprep)
    # Read the TSV file and convert to pandas dataframe
    confounds_file = glob(join(data_dir,
                               'sub-%s' % subject_id,
                               '*', 'func',
                               '*%s*confounds.tsv' % task))[0]
    regressors, regressor_names = process_confounds(confounds_file)
    
    # set up contrasts
    EV_dict = parse_EVs(events_df, task, regress_rt)
    
    subjectinfo = Bunch(conditions=EV_dict['conditions'],
                        onsets=EV_dict['onsets'],
                         durations=EV_dict['durations'],
                         amplitudes=EV_dict['amplitudes'],
                         tmod=None,
                         pmod=None,
                         regressor_names=regressor_names,
                         regressors=regressors.T.tolist())
    if inspect_inputs==True:
        regressors_df = pd.DataFrame(regressors, columns = regressor_names)
        return events_df, regressors_df
    else:
        contrasts = get_contrasts(task, regress_rt)
        return subjectinfo, contrasts  # this output will later be returned to infosource

def save_subjectinfo(base_directory, subject_id, task, subject_info, contrasts):
    from os import makedirs
    from os.path import join
    import pickle
    task_dir = join(base_directory, subject_id + '_task_' + task)
    makedirs(task_dir, exist_ok=True)
    subjectinfo_path = join(task_dir,'subjectinfo.pkl')
    pickle.dump(subject_info, open(subjectinfo_path,'wb'))
    
    contrast_path = join(task_dir,'contrasts.pkl')
    pickle.dump(contrasts, open(contrast_path,'wb'))
    return (subjectinfo_path, contrast_path)

View one events file used in subject info

In [4]:
bunch, contrasts = subjectinfo(data_dir,'s358','CCTHot',False)
events_df,confounds_df = subjectinfo(data_dir,'s358','CCTHot',False,True)
events_df.head() 

,onset,duration,response_time,EV,action,experiment_exp_id,gain_amount,gain_probability,key_press,loss_amount,loss_probability,num_cards,num_click_in_round,num_loss_cards,risk,total_cards,trial_type,worker_id,junk,feedback
0,3.009,1.137,1.172657,-1.410872,draw_card,columbia_card_task_fmri,12.0,0.833333,89.0,-70.0,0.166667,6.0,1.0,1.0,-1.872734,NaN,poldrack-single-stim,s358,False,True
1,5.588,1.137,-0.333343,-4.144206,draw_card,columbia_card_task_fmri,12.0,0.800000,89.0,-70.0,0.200000,6.0,2.0,1.0,0.367597,NaN,poldrack-single-stim,s358,False,True
2,6.659,1.137,-0.555343,-8.244206,draw_card,columbia_card_task_fmri,12.0,0.750000,89.0,-70.0,0.250000,6.0,3.0,1.0,3.074563,NaN,poldrack-single-stim,s358,False,True
3,7.510,1.137,0.806657,-15.077539,end_round,columbia_card_task_fmri,12.0,0.666667,71.0,-70.0,0.333333,6.0,4.0,1.0,6.222617,3.0,poldrack-single-stim,s358,False,True
4,12.179,1.137,1.226657,-0.577539,draw_card,columbia_card_task_fmri,14.0,0.833333,89.0,-75.0,0.166667,6.0,1.0,1.0,0.735928,NaN,poldrack-single-stim,s358,False,True


### Specify Input and Output Stream

In [5]:
# Get Subject Info - get subject specific condition information
getsubjectinfo = Node(Function(input_names=['data_dir', 'subject_id',
                                            'regress_rt', 'task'],
                               output_names=['subject_info', 'contrasts'],
                               function=subjectinfo),
                      name='getsubjectinfo')
getsubjectinfo.inputs.data_dir = data_dir
getsubjectinfo.inputs.regress_rt = regress_rt
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id',
                                            'task',
                                            'contrasts']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('task', task_list)]
# SelectFiles - to grab the data (alternative to DataGrabber)
templates = {'func': join('*{subject_id}','*','func',
                         '*{task}*MNI*preproc.nii.gz'),
            'mask': join('*{subject_id}','*','func',
                         '*{task}*MNI*brainmask.nii.gz')}
selectfiles = Node(SelectFiles(templates,
                               base_directory = data_dir,
                               sort_filelist=True),
                   name="selectfiles")
# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory = experiment_dir,
                         container=output_dir),
                name="datasink")
# Save python objects that aren't accomodated by datasink nodes
save_subjectinfo = Node(Function(input_names=['base_directory','subject_id',
                                              'task','subject_info','contrasts'],
                                 output_names=['output_path'],
                                function=save_subjectinfo),
                       name="savesubjectinfo")
save_subjectinfo.inputs.base_directory = join(experiment_dir,output_dir)
# Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                ('fstat', 'FSTST'),
                ('run0.mat', 'designfile.mat')]
datasink.inputs.substitutions = substitutions

### Model Specification

In [18]:
# mask and blur
masker = Node(fsl.maths.ApplyMask(),name='masker')

# SpecifyModel - Generates FSL-specific Model
modelspec = Node(SpecifyModel(input_units='secs',
                              time_repetition=TR,
                              high_pass_filter_cutoff=80),
                 name="modelspec")

# Level1Design - Generates an FSL design matrix
level1design = Node(fsl.Level1Design(bases={'dgamma':{'derivs': True}},
                                 interscan_interval=TR,
                                 model_serial_correlations=True),
                    name="level1design")

# FEATmodel
level1model = Node(fsl.FEATModel(), name="FEATModel")

# FILMGLs
# smooth_autocorr, check default, use FSL default
filmgls = Node(fsl.FILMGLS(), name="FILMGLS")

conestimate = Node(fsl.ContrastMgr(), name='conestimate',
                      iterfield=['tcon_file', 'param_estimates',
                                    'sigmasquareds', 'corrections',
                                    'dof_file'])

# Run as separate nodes

Useful for debugging

In [7]:
task = task_list[0]
subj = subject_list[0]

getsubjectinfo.inputs.subject_id = subj
getsubjectinfo.inputs.task = task
subject_info_out = getsubjectinfo.run()

selectfiles.inputs.subject_id = subj
selectfiles.inputs.task = task
selectfiles_out = selectfiles.run()


180103-08:31:36,939 workflow INFO:
	 Executing node getsubjectinfo in dir: /tmp/tmp35zrhmfp/getsubjectinfo
180103-08:31:36,946 workflow INFO:
	 Running node "getsubjectinfo" ("nipype.interfaces.utility.wrappers.Function").
180103-08:31:37,171 workflow INFO:
	 Executing node selectfiles in dir: /tmp/tmpzgh3_uyw/selectfiles
180103-08:31:37,173 workflow INFO:
	 Running node "selectfiles" ("nipype.interfaces.io.SelectFiles").


In [8]:
masker.inputs.in_file = selectfiles_out.outputs.func 
masker.inputs.mask_file = selectfiles_out.outputs.mask 
masker_out = masker.run()

modelspec.inputs.subject_info = subject_info_out.outputs.subject_info 
modelspec.inputs.functional_runs = masker_out.outputs.out_file 
modelspec_out = modelspec.run()

level1design.inputs.contrasts = subject_info_out.outputs.contrasts
level1design.inputs.session_info = modelspec_out.outputs.session_info 
level1design_out = level1design.run() 
level1design_out.outputs

level1model.inputs.fsf_file = level1design_out.outputs.fsf_files 
level1model.inputs.ev_files = level1design_out.outputs.ev_files 
out=level1model.run()

180103-08:31:40,987 workflow INFO:
	 Executing node masker in dir: /tmp/tmp4ye3crad/masker
180103-08:31:40,992 workflow INFO:
	 Running node "masker" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /mnt/OAK/sub-s358/ses-2/func/sub-s358_ses-2_task-stroop_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz -mas /mnt/OAK/sub-s358/ses-2/func/sub-s358_ses-2_task-stroop_run-1_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz /tmp/tmp4ye3crad/masker/sub-s358_ses-2_task-stroop_run-1_bold_space-MNI152NLin2009cAsym_preproc_masked.nii.gz.
180103-08:33:13,644 workflow INFO:
	 Executing node modelspec in dir: /tmp/tmple1nck23/modelspec
180103-08:33:13,696 workflow INFO:
	 Running node "modelspec" ("nipype.algorithms.modelgen.SpecifyModel").
180103-08:33:13,791 workflow INFO:
	 Executing node level1design in dir: /tmp/tmpwe4qdwab/level1design
180103-08:33:13,887 workflow INFO:
	 Running node "level1design" ("nipype.interfaces.fsl.model.Level1Design").
180103-0

In [9]:
filmgls.inputs.in_file = masker_out.outputs.out_file 
filmgls.inputs.design_file = out.outputs.design_file 
filmgls.inputs.tcon_file = out.outputs.con_file 
filmgls.inputs.fcon_file = out.outputs.fcon_file 
fil_out = filmgls.run()

180103-08:33:14,610 workflow INFO:
	 Executing node FILMGLS in dir: /tmp/tmpam5pihff/FILMGLS
180103-08:33:14,613 workflow INFO:
	 Running node "FILMGLS" ("nipype.interfaces.fsl.model.FILMGLS"), a CommandLine Interface with command:
film_gls --rn=results --con=/tmp/tmp86poicw7/FEATModel/run0.con --in=/tmp/tmp4ye3crad/masker/sub-s358_ses-2_task-stroop_run-1_bold_space-MNI152NLin2009cAsym_preproc_masked.nii.gz --pd=/tmp/tmp86poicw7/FEATModel/run0.mat --thr=0.000000.
180103-08:33:14,667 interface INFO:
	 stdout 2018-01-03T08:33:14.667203:Log directory is: results
180103-08:33:22,288 interface INFO:
	 stdout 2018-01-03T08:33:22.287950:paradigm.getDesignMatrix().Nrows()=325
180103-08:33:22,288 interface INFO:
	 stdout 2018-01-03T08:33:22.287950:paradigm.getDesignMatrix().Ncols()=29
180103-08:33:22,289 interface INFO:
	 stdout 2018-01-03T08:33:22.287950:sizeTS=325
180103-08:33:22,289 interface INFO:
	 stdout 2018-01-03T08:33:22.287950:numTS=177828
180103-08:33:22,330 interface INFO:
	 stdout 

In [24]:
conestimate.inputs.param_estimates = fil_out.outputs.param_estimates
conestimate.inputs.sigmasquareds = fil_out.outputs.sigmasquareds
conestimate.inputs.corrections = fil_out.outputs.corrections
conestimate.inputs.dof_file = fil_out.outputs.dof_file

conestimate.inputs.tcon_file = out.outputs.con_file

AttributeError: 'FILMGLSOutputSpec507' object has no attribute 'corrections'

In [10]:
datasink.inputs.copes = fil_out.outputs.copes
datasink.inputs.zstats = fil_out.outputs.zstats
datasink.inputs.tstats = fil_out.outputs.tstats
datasink.inputs.fstats = fil_out.outputs.fstats
datasink.inputs.param_estimates = fil_out.outputs.param_estimates
datasink.inputs.residual4d = fil_out.outputs.residual4d
datasink.inputs.sigmasquareds = fil_out.outputs.sigmasquareds

datasink.run()

180103-16:52:08,718 workflow INFO:
	 Executing node datasink in dir: /tmp/tmpzdxpmw7o/datasink
180103-16:52:08,730 workflow INFO:
	 Running node "datasink" ("nipype.interfaces.io.DataSink").


In [12]:
save_subjectinfo.inputs.subject_id = subj
save_subjectinfo.inputs.task = task
save_subjectinfo.inputs.subject_info = subject_info_out.outputs.subject_info
save_subjectinfo.inputs.contrasts = subject_info_out.outputs.contrasts

save_subjectinfo.run()

180103-16:52:35,964 workflow INFO:
	 Executing node savesubjectinfo in dir: /tmp/tmp7vam1ve4/savesubjectinfo
180103-16:52:36,35 workflow INFO:
	 Running node "savesubjectinfo" ("nipype.interfaces.utility.wrappers.Function").


# Workflow

In [ ]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = join(experiment_dir, working_dir)

# Connect up the 1st-level analysis components
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                               ('task', 'task')]),
                    (infosource, getsubjectinfo, [('subject_id','subject_id'),
                                                 ('task', 'task')]),
                    (selectfiles, masker, [('func','in_file'),
                                           ('mask', 'mask_file')]),
                    (getsubjectinfo, modelspec, [('subject_info','subject_info')]),
                    (masker, modelspec, [('out_file', 'functional_runs')]),
                    (modelspec, level1design, [('session_info','session_info')]),
                    (getsubjectinfo, level1design, [('contrasts','contrasts')]),
                    (level1design, level1model, [('ev_files', 'ev_files'),
                                                 ('fsf_files','fsf_file')]),
                    (masker, filmgls, [('out_file', 'in_file')]),
                    (level1model, filmgls, [('design_file', 'design_file'),
                                            ('con_file', 'tcon_file'),
                                            ('fcon_file', 'fcon_file')]),
                    (level1model, datasink, [('design_file', '1stLevel.@design_file')]),
                    (filmgls, datasink, [('zstats', '1stLevel.@Z'),
                                        ('fstats', '1stLevel.@F'),
                                        ('tstats','1stLevel.@T'),
                                        ('param_estimates','1stLevel.param_estimates')]),
                    (infosource, save_subjectinfo, [('subject_id','subject_id'),
                                                     ('task', 'task')]),
                    (getsubjectinfo, save_subjectinfo, [('subject_info','subject_info'),
                                                        ('contrasts','contrasts')]),
                    
                    ])


### Run the Workflow


In [ ]:
l1analysis.run('MultiProc', plugin_args={'n_procs': 4})

### Visualize Workflow

In [ ]:
# Create 1st-level analysis output graph
l1analysis.write_graph(graph2use='colored', format='png', simple_form=True)
# Visualize the graph
from IPython.display import Image
graph_file=join(l1analysis.base_dir, 'l1analysis', 'graph.dot.png')
Image(graph_file)


In [ ]:
try:
    shutil.move(graph_file, 'l1analysis')
except shutil.Error:
    pass
# remove working directory
shutil.rmtree(working_dir)

### Visualize Design Matrix

In [ ]:
!tree datasink/1stLevel/sub*


In [ ]:
design_file = glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_CCTHot','designfile.mat'))[0]
desmtx=numpy.loadtxt(design_file, skiprows=5)
plt.imshow(desmtx,aspect='auto',interpolation='nearest',cmap='gray')
plt.title('Design Matrix')
plt.show()
cc=numpy.corrcoef(desmtx.T)
plt.imshow(cc,aspect='auto',interpolation='nearest', cmap=plt.cm.viridis)
plt.colorbar()
plt.title('Correlation of Regressors')
plt.show()
"""
# show first regressors
plt.figure(figsize=[20,8])
plt.plot(desmtx[:,0],'b', label=subject_info.conditions[0])
plt.plot(desmtx[:,2],'r', label=subject_info.conditions[1])
for i in subject_info.onsets[0]:
    plt.axvline(i/.68, ymin=0, ymax=.1)
for i in subject_info.onsets[1]:
    plt.axvline(i/.68, ymin=0, ymax=.1, c='r')
plt.legend()
plt.title('Congruent/Incongruent Regressors')
"""

### Visualize Results

In [ ]:
from nilearn.plotting import plot_stat_map
from nilearn.image import smooth_img

anatimg = glob(join(experiment_dir,'Data','sub-s358','anat','*T1w*MNI*preproc*'))[0]
contrast_img = glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_ANT','zstat3.nii.gz'))[0]
plot_stat_map(smooth_img(contrast_img, 8), title='stroop effect',
              bg_img=anatimg, threshold=1, display_mode='z', cut_coords=(-30, -15, 0, 15, 30), dim=-1)

In [ ]:
import nilearn.plotting
import nilearn.image
import cPickle

for task in task_list:
    contrast_file = glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_' + task,'contrasts.pkl'))[0]
    contrasts = cPickle.load(open(contrast_file,'r'))
    contrasts = [contrast[0] for contrast in contrasts]
    contrast_imgs = sort(glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_' + task,'zstat*.nii.gz')))
    for i, contrast_img in enumerate(contrast_imgs):
        nilearn.plotting.plot_glass_brain(nilearn.image.smooth_img(contrast_img, 8),
                                          display_mode='lyrz', colorbar=True, plot_abs=False, threshold=0,
                                          title=contrasts[i])
